In [ ]:
## Aumentar datos con errores ortograficos

In [1]:
## Carga Datos

from utils import load_json

train_data = load_json("data/train_nlu.json")
test_data = load_json("data/test_nlu.json")

In [2]:
from misspelling_augmenter import MisspellingsAug
from nltk.corpus import stopwords
import random
import copy
from utils import save_json
## fixed random 
random.seed(42)

def augument_texts_intents(data, aug_pct = 0.6, tok_pct = 0.3):
    ## Aumentar errores otograficos

    ## Obtener stopwords
    stop_words = stopwords.words('spanish')

    stop_words.extend(["?",".", ",","¿"])

    question_words = ['qué','que','quién','quien', 'cuál','cual', 'dónde','donde', 'cuándo','cuando', 'cómo','como', 'por qué','por que','porqué','porque', 
    'cuánto','cuanto','cuantos']

    ## remover palabras de preguntas de stopwords
    stop_words = [ sw for sw in stop_words if not sw in question_words ]

    ## Generador de errores ortográficos

    misspellingsAug = MisspellingsAug(aug_pct, tok_pct)

    ## Train data augumenter misspelling

    intents = list(set(([sample['intent_name'] for sample in data])))

    aug_data = []

    for intent_name in intents:
        intent_examples = [ sample for sample in data if intent_name == sample['intent_name']]
        intent_texts = [ sample['text'] for sample in intent_examples]
        ## Aumentar con errores ortograficos
        aug_texts, words_replace_log = misspellingsAug.augument_texts(texts= intent_texts, stop_words = stop_words)
        intent_with_entities_examples = [ sample for sample in data if intent_name == sample['intent_name'] and len(sample['entities']) > 0]
        intent_with_entities_texts = [ sample['text'] for sample in intent_with_entities_examples]

        for aug_idx , aug_words_log in enumerate(words_replace_log):
            if len(aug_words_log) == 0:
                continue
            
            if aug_words_log['text'] in intent_with_entities_texts:
                idx_sample = intent_with_entities_texts.index(aug_words_log['text'])
                intent_sample  = intent_with_entities_examples[idx_sample]
                new_sample = copy.deepcopy(intent_sample)
                new_sample['text'] = aug_texts[aug_idx]
                
                entities = intent_sample['entities']
                new_entities = new_sample['entities']
                
                if len(entities) > 0:
                
                    for aug_log_word in aug_words_log['misspelling_words_replace']:
                        displace = 0
                        word = aug_log_word['word']
                        ms_word = aug_log_word['ms_word']
                        displace = displace + len(ms_word) - len(word)
                        for ent, new_ent in zip(entities, new_entities):
                            if ent['span_start'] > aug_log_word['span_word'][0]:
                                new_ent['span_start']  = displace + new_ent['span_start']
                                new_ent['span_end']  = displace + new_ent['span_end']
                            else:
                                if aug_log_word['span_word'][1] <= ent['span_end']:
                                    new_ent['span_end'] = displace + new_ent['span_end']
                    
                    for ent in new_entities:
                        ent['value'] = aug_texts[aug_idx][ent['span_start']:ent['span_end']]
                    
                    new_sample['entities'] = new_entities
                
                aug_data.append(new_sample)
                
            else: 
                new_sample =  {
                "text": aug_texts[aug_idx] ,
                "intent_name": intent_name,
                "entities": [],
                "text_label_entities": ""
                }
                aug_data.append(new_sample)

    return aug_data 

aug_pct = 0.6 # porcentaje de numero maximo en textos escogidos
tok_pct  = 0.3 # porcentaje de numero maximo de tokens reemplado en el texto

aug_train_data = augument_texts_intents(train_data, aug_pct, tok_pct)

aug_test_data = augument_texts_intents(test_data, aug_pct, tok_pct)

all_train_data = train_data + aug_train_data 
save_json(all_train_data, "data/aug_train_nlu")

all_test_data = test_data + aug_test_data
save_json(all_test_data, "data/aug_test_nlu")

tokens: ['Buenos', 'dias', ',', 'no', 'pude', 'conectarme', 'en', 'mi', 'horario', 'de', 'matricula']
idx2token: {'0': 'Buenos', '1': 'dias', '4': 'pude', '5': 'conectarme', '8': 'horario', '10': 'matricula'}
Insert Synthetic word: matricula
Insert Synthetic word: horario
tokens: ['perdi', 'mi', 'turno', 'de', 'matricula', 'paso', 'entonces', 'a', 'rezagado', '?']
idx2token: {'0': 'perdi', '2': 'turno', '4': 'matricula', '5': 'paso', '6': 'entonces', '8': 'rezagado'}
Insert Synthetic word: perdi
Insert Natural word: rezagado
tokens: ['Hola', ',', 'he', 'tenido', 'problemas', 'de', 'conexion', 'y', 'no', 'llegue', 'a', 'matricularme', 'que', 'debo', 'hacer']
idx2token: {'0': 'Hola', '4': 'problemas', '6': 'conexion', '9': 'llegue', '11': 'matricularme', '12': 'que', '13': 'debo', '14': 'hacer'}
Insert Natural word: que
Insert Natural word: hacer
Insert Synthetic word: Hola
tokens: ['Hola', ',', 'si', 'perdí', 'mi', 'turno', 'pasaría', 'ha', 'rezagado', 'entonces', '?']
idx2token: {'0': 